In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch 

from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, Dataset

from transformers import BertForSequenceClassification,  BertConfig, AdamW
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel

import time
import re

In [ ]:
import random
# Set the seed value all over the place to make this reproducible.  
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed=42)

# Get data

In [ ]:
path = '/kaggle/input/nlp-getting-started'
df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv' )
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv' )

In [ ]:
modelname = 'bert-base-uncased' #'google/electra-base-discriminator'# 
batchsize =40
tokenizer = AutoTokenizer.from_pretrained(modelname) 

# EDA

In [ ]:
def clean(text):
    text = text.lower()
    text=text.replace(r'&amp;?',r'and')
    text=text.replace(r'&lt;',r'<')
    text=text.replace(r'&gt;',r'>')
    
    res = re.sub(r'http(s)?:\/\/([\w\.\/])*' ,' ',text) # clean url http://x.x.x.x/xxx
    res = re.sub('[0-9]+', '', res) #  clean numbers
    res = re.sub(r'[!"#$%&()*+,-./:;=?@\\^_`"~\t\n\<\>\[\]\{\}]',' ',res) # clean special chars。
    res = re.sub(r'  +',' ',res) # convert multiple continues blank char to one blank char。
    return res.strip()

df['text'] = df['text'].apply(clean)
test_df['text'] =test_df['text'].apply(clean)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_sentence_embeddings_length(text_list, tokenizer):
    tokenized_texts = list(map(lambda t: tokenizer.tokenize(t), text_list))
    tokenized_texts_len = list(map(lambda t: len(t), tokenized_texts))
    fig, ax = plt.subplots(figsize=(8, 5));
    ax.hist(tokenized_texts_len, bins=40);
    ax.set_xlabel("Length of Comment Embeddings");
    ax.set_ylabel("Number of Comments");
    return max(tokenized_texts_len)

maxlen1 = plot_sentence_embeddings_length(df['text'].values, tokenizer)
maxlen2 = plot_sentence_embeddings_length(test_df['text'].values, tokenizer)
maxlen = max(maxlen1,maxlen2)
print(maxlen)

# Split dataset

In [ ]:
df = df.sample(frac=1.0)
train_df = df[:6500] 
valid_df = df[6500:] 
print(train_df.shape,valid_df.shape)

# Train element

In [ ]:
maxlen = 50
def prepare_input(text, keyword):
    keystr = str(keyword)
    inputs = tokenizer(text, #keystr, 
                           add_special_tokens=True,
                           max_length= maxlen,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs

# Package DataSet

In [ ]:
class TrainDataset(Dataset):
    def __init__(self,vdf):
        self.target = vdf['target'].values
        self.text = vdf['text'].values
        self.keyword = vdf['keyword'].values
        self.location = vdf['location'].values

    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        inputs = prepare_input( 
                               self.text[item], 
                               self.keyword[item])
        label = torch.tensor(self.target[item], dtype=torch.long)
        
        return (inputs['input_ids'],inputs['token_type_ids'],inputs['attention_mask'] ), label

# Combine the training inputs into a TensorDataset.
# dataset = TensorDataset(input_ids, attention_masks, labels)
train_ds = TrainDataset(train_df) 
valid_ds = TrainDataset(valid_df) 

In [ ]:
train_loader = DataLoader(train_ds,
                              batch_size=batchsize,
                              shuffle=True,
                              pin_memory=True, drop_last=False)
valid_loader = DataLoader(valid_ds,
                              batch_size=batchsize,
                              shuffle=True,
                              pin_memory=True, drop_last=False)

# Train Model

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    modelname, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

In [ ]:
# Tell pytorch to run this model on the GPU.
model.cuda()
print('the model is initialized')

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2.48e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 3 # 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_loader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train Process

In [ ]:
# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, (inputs, labels) in enumerate(train_loader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_loader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = inputs[0] .to(device)
        b_input_mask = inputs[1].to(device)
        b_labels = labels.to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        
        output= model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = output[0]
        logits =output[1]
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_loader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch,(inputs, labels) in enumerate(valid_loader):
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        b_input_ids = inputs[0] .to(device)
        b_input_mask = inputs[2].to(device)
        b_labels = labels.to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            output = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            loss = output[0]
            logits =output[1]
            
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(valid_loader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(valid_loader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

In [ ]:
print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

# Make Prediction

In [ ]:
class TestDataset(Dataset):
    def __init__(self,vdf):
        self.text = vdf['text'].values
        self.keyword = vdf['keyword'].values
        self.location = vdf['location'].values

    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        inputs = prepare_input( 
                               self.text[item], 
                               self.keyword[item])        
        
        return (inputs['input_ids'],inputs['token_type_ids'],inputs['attention_mask']) 

In [ ]:
test_ds = TestDataset(test_df)
test_loader = DataLoader(test_ds,
                              batch_size=batchsize,
                              shuffle=False,
                              pin_memory=True, drop_last=False)
predictions =np.array([]) 

In [ ]:
model.eval()

for batch,inputs in enumerate(test_loader):
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        b_input_ids = inputs[0] .to(device)
        b_input_mask = inputs[2].to(device)

        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():   
            output = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask
                                   )
            logits =output[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        # y_pred_label = logits.argmax(dim=1)
        pred_flat = np.argmax(logits, axis=1).flatten()
        predictions = np.concatenate([predictions,pred_flat])

In [ ]:
submission_df = {"id":test_df['id'],
                 "target":predictions.astype(int)}
submission = pd.DataFrame(submission_df)
submission.to_csv('submission.csv',index=False)